
- [article on dijkstra](https://medium.com/basecs/finding-the-shortest-path-with-a-little-help-from-dijkstra-613149fbdc8e)
- [video on dijkstra](https://www.youtube.com/watch?v=pVfj6mxhdMw)


In [244]:
depth=7863
elf_x=14
elf_y=760

depth=510
elf_x=10
elf_y=10

# Part 1
build the intial map of where every thing is.

In [245]:
geo_index = [[0 for i in range(elf_x+6)] for j in range(elf_y+6)]
ero_index = [[0 for i in range(elf_x+6)] for j in range(elf_y+6)]

t = ['.', '=', '|']

for x in range(0, elf_x+6):
    for y in range(0, elf_y+6):
        if (x==elf_x) and (y==elf_y):
            geo_index[y][x] = 0
        elif x==0 and y==0:
            geo_index[y][x] == 0
        elif y == 0:
            geo_index[y][x] = x * 16807
        elif x == 0:
            geo_index[y][x] = y * 48271
        else:
            geo_index[y][x] = ero_index[y][x-1] * ero_index[y-1][x]

        ero_index[y][x] =  ((geo_index[y][x] + depth) % 20183)

for x in range(0, elf_x+6):
    for y in range(0, elf_y+6):
        ero_index[y][x] = ero_index[y][x] % 3

# Part 2
build a node graph 

In [246]:
import networkx as nx
import matplotlib
import matplotlib.pyplot as plt

%matplotlib inline 
G = nx.Graph()
unvisited = []
visited = []

nodes = {}
edges = {}
graph = {}

In [247]:
#
# Find the smallest unvisisted node
#
def min_unvisited_node(graph):
    min_v = 99999
    m_node = -1
    for node in graph.keys():
        # if the node is smaller than the min, and is unvisited 
        if graph[node][0] < min_v and graph[node][3] == False:
            min_v = graph[node][0]
            m_node = node
    return m_node

rock = 0
wet = 1
narrow = 2

climbing_gear = 2
torch = 1
neither = 0

shortest_path_to_a = 0
prev_node = 1
tool = 2
visited = 3

eindex = 0
node_neighbors = 1

valid_tools = [ [climbing_gear,torch], 
                [climbing_gear,neither], 
                [torch,neither] 
              ]

In [256]:
for x in range(0, elf_x+5):
    for y in range(0, elf_y+5):
        neigh = []
        for dir in [ (0,1), (0,-1), (1,0), (-1,0)]:
            if x + dir[0] > elf_x+4 or x+dir[0] < 0 or y+dir[1] > elf_y+4 or y+dir[1] < 0:
                continue
            neigh.append( str(x+dir[0]) + "_" + str(y+dir[1]) )
        
        # (shortest_path_a, prev, tool, visisted)
        graph[str(x) + "_" + str(y)] = [99999, False, False, False]
        nodes[str(x) + "_" + str(y)] = [ero_index[y][x],neigh]

graph['0_0'][shortest_path_to_a] = 0
graph['0_0'][tool] = torch # start with a torch

while True:
    # get the current smallest node that isn't visited 
    current_min_node = min_unvisited_node(graph)

    #px,py = map(int, current_min_node.split("_"))
    #if px == elf_x and py == elf_y:
    #    break
        
    # and we are done
    if current_min_node == -1:
        break

    # get the distance from the current_min_node to all it's neighbors 
    for n in  nodes[current_min_node][node_neighbors]:
        current_tool = graph[ current_min_node ][tool]
        (x,y) = map(int, n.split('_'))

            
        if current_tool in valid_tools[ ero_index[y][x]]:
            # current tools can be used in this area
            points = 1
            eq = current_tool
        else:
            # current tool must change
            points = 8
            if nodes[n][eindex] == rock and nodes[current_min_node][eindex] == wet:
                eq = climbing_gear
            elif nodes[n][eindex] == rock and nodes[current_min_node][eindex] == narrow:
                eq = torch
            elif nodes[n][eindex] == wet and nodes[current_min_node][eindex] == rock:
                eq = climbing_gear
            elif nodes[n][eindex] == wet and nodes[current_min_node][eindex] == narrow:
                eq = neither
            elif nodes[n][eindex] == narrow and nodes[current_min_node][eindex] == rock:
                eq = torch
            elif nodes[n][eindex] == narrow and nodes[current_min_node][eindex] == wet:
                eq = neither    
            
        t = graph[current_min_node][shortest_path_to_a] + points
        if graph[n][shortest_path_to_a] > t:
            graph[n][shortest_path_to_a] = t
            graph[n][tool] = eq
            graph[n][prev_node] = current_min_node

    graph[current_min_node][visited] = True
    

In [257]:

d = graph[ str(elf_x) + "_" + str(elf_y)]

z = str(elf_x) + "_" + str(elf_y)
out = []
while z != '0_0':
    out.append(z)
    z = graph[z][prev_node]

out.append('0_0')
while len(out) >0:
    j = out.pop()
    print( j, graph[j] )

print( d[shortest_path_to_a])


0_0 [0, False, 1, True]
0_1 [1, '0_0', 1, True]
0_2 [2, '0_1', 1, True]
0_3 [10, '0_2', 2, True]
0_4 [11, '0_3', 2, True]
0_5 [12, '0_4', 2, True]
1_5 [20, '0_5', 0, True]
2_5 [21, '1_5', 0, True]
2_6 [22, '2_5', 0, True]
2_7 [30, '2_6', 2, True]
2_8 [31, '2_7', 2, True]
2_9 [32, '2_8', 2, True]
3_9 [33, '2_9', 2, True]
4_9 [34, '3_9', 2, True]
5_9 [35, '4_9', 2, True]
5_10 [36, '5_9', 2, True]
5_11 [37, '5_10', 2, True]
5_12 [38, '5_11', 2, True]
6_12 [39, '5_12', 2, True]
7_12 [40, '6_12', 2, True]
8_12 [41, '7_12', 2, True]
9_12 [42, '8_12', 2, True]
10_12 [43, '9_12', 2, True]
10_11 [44, '10_12', 2, True]
10_10 [45, '10_11', 2, True]
45
